Para aplicação do algoritmo do problema do caixeiro viajante, usamos como caso de uso o serviço de transporte de empregados da empresa estatal Nuclebras Equipamentos Pesados (NUCLEP). Esta empresa estatal do governo federal é vinculada ao Ministério de Minas e Energia. A NUCLEP tem como objetivos principais projetar, desenvolver, fabricar e comercializar equipamentos pesados para os setores Nuclear, Defesa, Óleo e Gás, Energia e outros. Estratégica pela tecnologia que domina, possui características únicas de suas instalações e equipamentos. Possui uma área de 1.5 milhão de metros quadrados, com 85 mil metros quadrados de área fabril. A NUCLEP está geograficamente localizada no município de Itaguaí, no Estado do Rio de Janeiro. Possui também Terminal Portuário Privativo de embarque para até 39 ton/m2. Conta com cerca de 800 empregados diretos, sem contar a mão-de-obra terceirizada. A empresa oferece transporte para os seus colaboradores em vários pontos no estado do Rio de Janeiro. Contando com 30 linhas de transporte regular que cobrem várias cidades e bairros da capital fluminense. Ainda assim, durante alguns projetos, são necessários deslocamentos de empregados em outros horários. Nesse ponto, a programação da logística de transporte da empresa, deve distribuir transporte para vários empregados a semelhança do problema do caixeiro viajante. Ou seja, a partir da rodoviária da NUCLEP os transportes partem para vários pontos do estado e retornam ao ponto de origem para os empregados iniciarem suas atividades. 

Como objeto de estudo foi selecionado uma amostragem a partir de uma das linhas. Apesar de contar com 30 linhas regulares, totalizando 575 pontos de embarque, esta segmentação ainda permite uma otimização do transporte oferecido pela empresa. Para isso foi utilizada a linha 11, de Jacarepaguá, com 25 pontos de embarque, para os testes dos algoritmos. Esta linha em questão atende não somente Jacarepaguá, mas os bairros Anil, Barra da Tijuca, Freguesia, Pechincha, Sulacap, Tanque e Taquara.

Pré-processamento

Os dados de embarque informados pela NUCLEP não possuíam informações de georeferenciadas. Com isso foi necessário um trabalho de pré-processamento para tratar esta informação no modelo de algoritmo de caixeiro viajante. Foi utilizada a biblioteca de funções Geoapify para determinar as referências geográficas. Durante esta fase ainda foi necessário um refinamento nos pontos de dados informados, pois a base disponibilizada pela empresa não indicava, em alguns pontos, o número do logradouro, usando referências de construções, por proximidade, para facilitar o entendimento do empregado em relação ao ponto de embarque.

In [27]:
import pandas as pd
import requests

# Função para retornar a coordenada de um endereço
def retorne_coordenada(endereco):
    api_key = 'cf30db6de8bf4244b1626a7688b31701'
    url = f'https://api.geoapify.com/v1/geocode/search?text={endereco}&apiKey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
    return None

# Carregar os pontos de embarque da Linha de jacarepaguá (código 55)
pontos_linha_jpa = pd.read_csv("pontos_linha_jpa.csv", sep=";", header=0, encoding="iso-8859-1")

A partir da função retorne_coordenada os dados dos pontos de embarque passaram a contar com os dados de longitude e latitude. Além de adicionar o ponto de origem da NUCLEP com o primeiro ponto do dataset criado.

In [28]:
# Adicionar o endereço da Nuclep ao dataset pontos_linha_jpa
nuclep_endereco = "NUCLEP"

# Criar um DataFrame com o endereço da Nuclep
df_nuclep = pd.DataFrame({'nmPonto': [nuclep_endereco]})

# Concatenar o DataFrame df_nuclep com pontos_linha_jpa
pontos_linha_jpa = pd.concat([df_nuclep, pontos_linha_jpa], ignore_index=True)

# Loop para ler cada registro e atribuir valores de latitude e longitude
for index, row in pontos_linha_jpa.iterrows():
    coordenadas = retorne_coordenada(row['nmPonto'])
    print(row['nmPonto'])
    print(coordenadas)
    if coordenadas:
        pontos_linha_jpa.at[index, 'latitude'] = coordenadas[1]
        pontos_linha_jpa.at[index, 'longitude'] = coordenadas[0]
    else:
        pontos_linha_jpa.at[index, 'latitude'] = None
        pontos_linha_jpa.at[index, 'longitude'] = None

NUCLEP
[-43.8288403, -22.8934134]
Avenida Geremario Dantas, 1083
[-43.3514921, -22.9365601]
Estrada de Jacarepagua, 7300
[-43.3389721, -22.9455302]
Estrada do Tindiba, 1148
[-43.3651905, -22.9284232]
Estrada dos Tres Rios, 223
[-43.3416453, -22.9392088]
Rua Marechal Jose Bevilaqua, 240
[-43.36932, -22.923261]
Rua Tirol, 356
[-43.3385247, -22.940199]
Rua Bacairis, 106
[-43.372278, -22.920207]
Avenida dos Mananciais, 771
[-43.3855556, -22.9170584]
Avenida Alberico Diniz, 1645
[-43.392382, -22.886949]
Estrada do Engenho Dagua, 2
[-43.349606, -22.955552]
Estrada do Catonho, 650
[-43.3798223, -22.9070657]
Estrada Coronel Pedro Correa, 1470
[-43.3872686, -22.9717692]


A parte final do pré-processamento é a criação de uma matriz de distância. Onde cada linha na matriz representa um dos pontos de embarque e o ponto de origem, normalmente indentificado como depot=0 (zero). E no par linha/coluna nesta matriz são armazenadas as distâncias, o que infere que a diagonal principal da matriz apresente todos os valores zerados. Para fazer esta matriz foi utilizado uma função do geoapify routematrix. Para atender ao formato de entrada desta função, os dados de longitude e latitude foram consolidados em uma matriz coordenadas.

In [29]:
# Função para calcular a distância entre os pontos de embarque e gerar a matriz de distâncias
def calcular_distancias(coordenadas):
    api_key = 'cf30db6de8bf4244b1626a7688b31701'
    url = 'https://api.geoapify.com/v1/routematrix?apiKey=' + api_key
    payload = {
        "mode": "drive",
        "sources": [{"location": coord} for coord in coordenadas],
        "targets": [{"location": coord} for coord in coordenadas]
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        data = response.json()
        if 'sources_to_targets' in data:
            distancias = []
            for linha in data['sources_to_targets']:
                distancias.append([item['distance'] for item in linha])
            return distancias
    return None

def criar_modelo(): 
    # Montar um vetor de coordenadas com o par latitude e longitude vindo do dataset pontos_linha_jpa
    coordenadas = []
    for index, row in pontos_linha_jpa.iterrows():
        if pd.notnull(row['longitude']) and pd.notnull(row['latitude']):
            coordenadas.append([row['longitude'], row['latitude']])
            
    # Criar a matriz de distâncias
    matriz_distancia = calcular_distancias(coordenadas)

    # Exibir a matriz de distância quebrando em nova linha cada conjunto
    for linha in matriz_distancia:
        print(linha)
        
    return {"distance_matrix": matriz_distancia, "depot": 0}    

Para a simulação de um algoritmo que atenda ao problema do caixeiro viajante, faremos uma abordagem inicial em força bruta. Nesta estratégia deve-se considerar todas as permutações possíveis dos pontos do itinerário, representadas pelos pontos de embarque, sejam analisadas. Reconhecidamente por este método o custo computacional pode ser exponencial, forçando limitarmos o número de pontos usados na iteração. Nos testes executados, tivemos um desempenho aceitável até 13 pontos de embarque, acima deste valor o algoritmo ficou inviável.

<descrever a relação matemática de uso exponencial de recursos que demonstre a restrição ao algoritmo de força bruta>

Primeiramente é definida a função calculate_route_distance que incrementa as distâncias dos pontos a partir de uma rota proposta passada em parâmetro, retornando o total percorrido incluindo o ponto de origem no início e o retorno a origem no final da rota.  

In [30]:
from itertools import permutations
import time

def create_data_model():
    """Stores the data for the problem with the specified number of cities."""
    distance_matrix = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]

    #distance_matrix = data
    #distance_matrix = [[0, 19072, 25995], [20314, 0, 8390], [29115, 8816, 0]]
          
    return {"distance_matrix": distance_matrix, "depot": 0}

def calculate_route_distance(route, distance_matrix):
    """Calculates the total distance of a given route."""
    distance = 0
    for i in range(len(route) - 1):
        distance += distance_matrix[route[i]][route[i + 1]]
    # Add return to the depot
    distance += distance_matrix[route[-1]][route[0]]
    return distance

Para o algoritmo de força bruta propriamente, a função brute_force_tsp executa um procedimento iterativo, a partir da matriz de distância, com todas as permutações de cada um dos pontos de embarque. Para a permutação foi utilizado a biblioteca permutations do Python. Com isso é calculado a distância de cada uma dessas rotas (usando a função calculate_route_distance), comparando os resultados obtidos até determinar a rota com a menor distância. A função time() contabiliza o tempo de processamento.

In [ ]:
def brute_force_tsp(data):
    """Solve the TSP using brute force."""
    num_cities = len(data["distance_matrix"])
    depot = data["depot"]
    cities = list(range(num_cities))
    cities.remove(depot)  # Exclude the depot from the permutations

    min_distance = float("inf")
    best_route = None

    for perm in permutations(cities):
        current_route = [depot] + list(perm) + [depot]  # Start and end at depot
        current_distance = calculate_route_distance(current_route, data["distance_matrix"])
        if current_distance < min_distance:
            min_distance = current_distance
            best_route = current_route

    return best_route, min_distance

def main():
    
    #data = create_data_model()
    data = criar_modelo()
    
    # Medir o tempo inicial
    start_time = time.time()

    best_route, min_distance = brute_force_tsp(data)
    
    # Medir o tempo final
    end_time = time.time()
    execution_time = end_time - start_time

    print(f"Melhor rota: {' -> '.join(map(str, best_route))}")
    print(f"Distância mínima: {min_distance} metros")
    print(f"Tempo de execução: {execution_time:.2f} segundos")


#if _name_ == "_main_":
main()

[0, 66247, 68609, 63384, 68761, 62987, 68349, 62067, 60015, 55725, 72007, 59450, 68050]
[65338, 0, 2362, 2517, 3033, 3245, 2102, 4594, 6422, 10300, 4284, 6024, 7050]
[68355, 3020, 0, 5537, 1877, 6265, 946, 7614, 9442, 13317, 1818, 9044, 7461]
[62270, 2129, 4491, 0, 4643, 728, 4231, 1253, 3081, 8027, 6413, 3748, 9179]
[66478, 1143, 1608, 3660, 0, 4388, 1348, 5737, 7565, 11440, 2530, 7167, 7833]
[61680, 4204, 6566, 1133, 6718, 0, 6306, 525, 2353, 7016, 8488, 2737, 11254]
[67409, 2074, 1248, 4591, 931, 5319, 0, 6668, 8496, 12371, 3461, 8098, 8764]
[61529, 3583, 5945, 1454, 6097, 1055, 5685, 0, 1742, 6491, 7867, 2212, 10633]
[59215, 4914, 7276, 2785, 7428, 2386, 7016, 2850, 0, 6182, 9198, 1838, 12302]
[55503, 10589, 12951, 8063, 13103, 7665, 12691, 6749, 5946, 0, 14873, 4131, 15936]
[72478, 3174, 2996, 5691, 3667, 6419, 2736, 7768, 9596, 13474, 0, 9198, 6263]
[60316, 5805, 8167, 3938, 8319, 3539, 7907, 2618, 2219, 5276, 10089, 0, 12855]
[65422, 9347, 11525, 7222, 12196, 6823, 11265, 7348, 